In [1]:
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value,
                           units as pyunits)

import copy

from pyomo.network import Arc, SequentialDecomposition

from idaes.core.util.initialization import propagate_state

# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

import idaes.logger as idaeslog
from pyomo.opt import TerminationCondition, SolverStatus

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

# Import the Generic Parameter Block
from idaes.models.properties.modular_properties.base.generic_property import (
        GenericParameterBlock)

# Import unit models from the model library
from idaes.generic_models.unit_models import Flash

# Import R32, R125, IL info file
from HFCs_bmimPF6_PR_m import configuration

    has been moved to idaes.models.properties.modular_properties.phase_equil
    (deprecated in 2.0.0.alpha0) (called from <frozen
    importlib._bootstrap>:241)


In [3]:
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.testing import initialization_tester
from idaes.core.util import scaling as iscale
from idaes.core.solvers import get_solver

In [5]:
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.props = GenericParameterBlock(default=configuration)

m.fs.F101 = Flash(default={"property_package": m.fs.props,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

m.fs.F102 = Flash(default={"property_package": m.fs.props,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

m.fs.F103 = Flash(default={"property_package": m.fs.props,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

m.fs.props.PR_kappa['R32', 'R125'].fix(0.009)
m.fs.props.PR_kappa['R125', 'R32'].fix(0.007)
m.fs.props.PR_kappa['R32', 'bmimPF6'].fix(-0.02608)
m.fs.props.PR_kappa['bmimPF6', 'R32'].fix(-0.0704)
m.fs.props.PR_kappa['bmimPF6', 'R125'].fix(0.349073)
m.fs.props.PR_kappa['R125', 'bmimPF6'].fix(0.058856)


2022-06-29 20:34:08 [WARNING] idaes.core.base.unit_model: DEPRECATION: <class 'idaes.core.base.process_block._ScalarPackedColumn'> has overloaded the initialize method. In v2.0, IDAES Will be moving to having a centralized initialize method which calls unit-specific initialize_build methods instead. Model developers should update their models to implement the initialize_build method instead of overloading initialize.


In [6]:
print(degrees_of_freedom(m))

200
